Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать 

## 1. объединить в одну выборку

In [ ]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-04-25 13:57:37--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-04-25 13:57:40--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd6bc6ab4e5ce5d79c56bbff405.dl.dropboxusercontent.com/cd/0/inline/BkFqRCC-mi4-BlK2TIsu7dtim-BNoVrio5dabKtuMllmPtDEV0jJ6bTrraOwDSAyKmONvzGF0mVNKSaqddC0IVuT0pX2i-ppM2vYiGAq57wnWSy0Ki4JlNs3iOXXR0ADX3giR6ZoOgRklZmQ1W79O3GwAzGuzXuduLzjIKusfOCY1w/file# [following]
--2022-04-25 13:57:40--  https://ucd6bc6ab4e5ce5d79c56bbff405.dl.dropboxusercontent.com/cd/0/inline/BkFqRCC-mi4-BlK2TIsu7dtim-BNoVrio5dabKtuMllmPtDEV0jJ6bTrraOwDSAyKmON

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import re
import nltk

In [ ]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install stop_words

In [ ]:
!pip install annoy

In [ ]:
def preprocess_txt(line):
  morpher = MorphAnalyzer()
  sw = set(get_stop_words("ru"))
  exclude = set(string.punctuation)
  
  spls = "".join(i for i in line.strip() if i not in exclude).split()
  spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
  spls = [i for i in spls if i not in sw and i != ""]
  return spls

In [ ]:
def get_response(question, index, model, index_map):
  question = preprocess_txt(question)
  vector = np.zeros(300)
  norm = 0
  for word in question:
    if word in model.wv:
      vector += model.wv[word]
      norm += 1
  if norm > 0:
    vector = vector / norm
  answers = index.get_nns_by_vector(vector, 3, )
  return [index_map[i] for i in answers]

In [ ]:
TEXT = "Какой город самый красивый?"

In [ ]:
preprocess_txt(TEXT)

['город', 'самый', 'красивый']

In [ ]:
df.head(3)

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/nlp_tut/lesson3/"

In [ ]:
def lower_words(text):
  lower_text = text.lower()

  return lower_text

In [ ]:
df['text'] = df['text'].apply(lower_words)

In [ ]:
df.head(5)

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"да, все-таки он немного похож на него. но мой ...",positive
2,rt @katiacheh: ну ты идиотка) я испугалась за ...,positive
3,"rt @digger2912: ""кто то в углу сидит и погибае...",positive
4,@irina_dyshkant вот что значит страшилка :d\nн...,positive


In [ ]:
def some_del(text, dictionary):
  """ deleting something, if it in dictionary """
  
  apost_del_text = text
  for word in text.split():
    #print(f"word in text {word}")
    if word in dictionary:
      #print(f"word for deleting {word}")
      #apost_del_text = re.sub(r'word', dictionary[word], apost_del_text)
      apost_del_text = re.sub(f'{word}', dictionary[word], apost_del_text)
      #print(f"word after deleting {dictionary[word]}")
      

  return apost_del_text

In [ ]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [ ]:
df['text'] = df['text'].apply(some_del, dictionary = apostrophe_dict)

In [ ]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"да, все-таки он немного похож на него. но мой ...",positive
2,rt @katiacheh: ну ты идиотка) я испугалась за ...,positive
3,"rt @digger2912: ""кто то в углу сидит и погибае...",positive
4,@irina_dyshkant вот что значит страшилка :d\nн...,positive


In [ ]:
df['text'] = df['text'].apply(some_del, dictionary = short_word_dict)

In [ ]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"да, все-таки он немного похож на него. но мой ...",positive
2,rt @katiacheh: ну ты идиотка) я испугалась за ...,positive
3,"rt @digger2912: ""кто то в углу сидит и погибае...",positive
4,@irina_dyshkant вот что значит страшилка :d\nн...,positive


In [ ]:
def replace_apostrophe(text, dictionary):
  words = [dictionary[word] if word in list(dictionary.keys()) else word for word in text.split()]
  return ' '.join(words)

In [ ]:
df['text'] = df['text'].apply(replace_apostrophe, dictionary = emoticon_dict)

In [ ]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"да, все-таки он немного похож на него. но мой ...",positive
2,rt @katiacheh: ну ты идиотка) я испугалась за ...,positive
3,"rt @digger2912: ""кто то в углу сидит и погибае...",positive
4,@irina_dyshkant вот что значит страшилка :d но...,positive


In [ ]:
def pattern_del(text, pattern):
  """ deleting patern mathing from text """
  
  no_pattern = re.sub(pattern, ' ', text)
  #print(no_pattern)

  return no_pattern

In [ ]:
df['text'] = df['text'].apply(pattern_del, pattern = r'[^\w\s]')

In [ ]:
df.head()

,text,label
0,first_timee хоть я и школота но поверь у на...,positive
1,да все таки он немного похож на него но мой ...,positive
2,rt katiacheh ну ты идиотка я испугалась за ...,positive
3,rt digger2912 кто то в углу сидит и погибае...,positive
4,irina_dyshkant вот что значит страшилка d но...,positive


In [ ]:
df.head()

,text,label
0,first_timee хоть я и школота но поверь у на...,positive
1,да все таки он немного похож на него но мой ...,positive
2,rt katiacheh ну ты идиотка я испугалась за ...,positive
3,rt digger2912 кто то в углу сидит и погибае...,positive
4,irina_dyshkant вот что значит страшилка d но...,positive


In [ ]:
#df['text'] = df['text'].apply(pattern_del, pattern = r'[^a-zA-Z]')

In [ ]:
def one_letter_word(text):
  text_new =  ' '.join([w for w in text.split() if len(w)>1])
  return text_new

In [ ]:
df['text'] = df['text'].apply(one_letter_word)

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df['text_token'] = df['text'].apply(tknz.word_tokenize)

In [ ]:
df.head(10)

,text,label,text_token
0,first_timee хоть школота но поверь нас то же с...,positive,"[first_timee, хоть, школота, но, поверь, нас, ..."
1,да все таки он немного похож на него но мой ма...,positive,"[да, все, таки, он, немного, похож, на, него, ..."
2,rt katiacheh ну ты идиотка испугалась за тебя,positive,"[rt, katiacheh, ну, ты, идиотка, испугалась, з..."
3,rt digger2912 кто то углу сидит погибает от го...,positive,"[rt, digger2912, кто, то, углу, сидит, погибае..."
4,irina_dyshkant вот что значит страшилка но бли...,positive,"[irina_dyshkant, вот, что, значит, страшилка, ..."
5,ну любишь или нет не знаю кто ты бля http co b...,positive,"[ну, любишь, или, нет, не, знаю, кто, ты, бля,..."
6,rt spoonlamer ох 900 ну это конечно же twinkle...,positive,"[rt, spoonlamer, ох, 900, ну, это, конечно, же..."
7,rt veregijytaqo тебя есть ухажёр нет мои уши н...,positive,"[rt, veregijytaqo, тебя, есть, ухажёр, нет, мо..."
8,поприветствуем моего нового читателя alexey1789,positive,"[поприветствуем, моего, нового, читателя, alex..."
9,теперь меня есть частичка сиднея happy sydney ...,positive,"[теперь, меня, есть, частичка, сиднея, happy, ..."


In [ ]:
#df['text_token'] = df['text'].apply(preprocess_txt)

In [ ]:
df.to_pickle(path + 'df1.pkl')

In [ ]:
sentenses = [row.split() for row in df['text']]

In [ ]:
sentenses[0]

['first_timee',
 'хоть',
 'школота',
 'но',
 'поверь',
 'нас',
 'то',
 'же',
 'самое',
 'общество',
 'профилирующий',
 'предмет',
 'типа']

In [ ]:
Word2Vec

gensim.models.word2vec.Word2Vec

In [ ]:
modelW2V_text = Word2Vec(sentences=sentenses, size = 300, window=5, min_count = 1)

In [ ]:
w2v_index = annoy.AnnoyIndex(300, 'angular')

index_map = {}
counter = 0

for line in df['text_token']:
  n_w2v = 0
  index_map[counter] = line[1]


IndexError: ignored

In [ ]:
text = 'поприветствуем моего нового читателя'

In [ ]:
text = preprocess_txt(text)


In [ ]:
text

['поприветствовать', 'новый', 'читатель']

In [ ]:
get_response(text, w2v_index, modelW2V, index_map)

NameError: ignored

In [ ]:
modelW2V.wv.most_similar(positive=text)

KeyError: ignored